<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bs4 as bs
import urllib.request
import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [39]:
url = "https://en.wikipedia.org/wiki/How_I_Met_Your_Mother"

In [65]:
raw_html = urllib.request.urlopen(url)
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [48]:
# [article for article in article_text.split("\n") if len(article) > 1]
df = pd.DataFrame([article for article in article_text.split("\n") if len(article.split(" ")) > 3], columns=['data'])
df.head()

,data
0,how i met your mother (often abbreviated as hi...
1,the series was loosely inspired by thomas and ...
2,"known for its unique structure, humor, and inc..."
3,the series follows the adventures of ted mosby...
4,the story goes into a flashback and starts in ...


In [49]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 89


### 1 - Preprocesamiento

In [50]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [51]:
# Demos un vistazo
sentence_tokens[:2]

[['how',
  'i',
  'met',
  'your',
  'mother',
  'often',
  'abbreviated',
  'as',
  'himym',
  'is',
  'an',
  'american',
  'sitcom',
  'created',
  'by',
  'craig',
  'thomas',
  'and',
  'carter',
  'bays',
  'for',
  'cbs',
  'the',
  'series',
  'that',
  'aired',
  'from',
  'september',
  '19',
  '2005',
  'to',
  'march',
  '31',
  '2014',
  'follows',
  'the',
  'main',
  'character',
  'ted',
  'mosby',
  'and',
  'his',
  'group',
  'of',
  'friends',
  'in',
  'new',
  'york',
  "city's",
  'manhattan',
  'as',
  'a',
  'framing',
  'device',
  'ted',
  'in',
  '2030',
  'recounts',
  'to',
  'his',
  'son',
  'luke',
  'and',
  'daughter',
  'penny',
  'the',
  'events',
  'from',
  'september',
  '2005',
  'to',
  'may',
  '2013',
  'that',
  'led',
  'him',
  'to',
  'meet',
  'their',
  'mother',
  'how',
  'i',
  'met',
  'your',
  'mother',
  'is',
  'a',
  'joint',
  'production',
  'by',
  'bays',
  'thomas',
  'productions',
  'and',
  '20th',
  'century',
  'fox'

### 2 - Crear los vectores (word2vec)

In [52]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [53]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=2,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=200,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [54]:
# Buildear el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [55]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 89


In [56]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 879


### 3 - Entrenar el modelo generador

In [57]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 164839.59375
Loss after epoch 1: 70111.15625
Loss after epoch 2: 60457.46875
Loss after epoch 3: 58956.25
Loss after epoch 4: 58830.3125
Loss after epoch 5: 57488.84375
Loss after epoch 6: 58541.75
Loss after epoch 7: 58759.8125
Loss after epoch 8: 58196.875
Loss after epoch 9: 59341.75
Loss after epoch 10: 59308.625
Loss after epoch 11: 59135.375
Loss after epoch 12: 60784.0625
Loss after epoch 13: 59779.75
Loss after epoch 14: 58644.875
Loss after epoch 15: 57086.0
Loss after epoch 16: 50412.375
Loss after epoch 17: 51567.375
Loss after epoch 18: 51057.875
Loss after epoch 19: 51164.125


(97977, 168920)

### 4 - Ensayar

In [59]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["marshall"], topn=10)

[('lily', 0.9981820583343506),
 ('robin', 0.9980660676956177),
 ('as', 0.9966107606887817),
 ('ted', 0.996426522731781),
 ('to', 0.9962630271911621),
 ('two', 0.9959754943847656),
 ('decide', 0.9956697225570679),
 ('in', 0.9955923557281494),
 ('job', 0.9954426884651184),
 ('and', 0.994999885559082)]

In [60]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["marshall"], topn=10)

[('your', -0.5273457765579224),
 ('i', -0.5553606152534485),
 ('met', -0.5586082935333252),
 ('how', -0.5744413733482361),
 ('mother', -0.6313814520835876),
 ('father', -0.7659263610839844),
 ('dad', -0.8653206825256348),
 ('would', -0.9082147479057312),
 ('story', -0.9176828861236572),
 ('he', -0.9204344749450684)]

In [61]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["met"], topn=10)

[('i', 0.9962379336357117),
 ('how', 0.996040940284729),
 ('mother', 0.9943609833717346),
 ('your', 0.9941858649253845),
 ('father', 0.9596943259239197),
 ('dad', 0.8978784084320068),
 ('would', 0.8492344617843628),
 ('story', 0.8394259214401245),
 ('he', 0.8350350260734558),
 ('does', 0.8269525766372681)]

In [62]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mother"], topn=10)

[('met', 0.9943609237670898),
 ('i', 0.9940323233604431),
 ('how', 0.992950439453125),
 ('your', 0.9872229099273682),
 ('father', 0.9817196130752563),
 ('dad', 0.9340349435806274),
 ('would', 0.8927876949310303),
 ('story', 0.8853969573974609),
 ('he', 0.8821658492088318),
 ('does', 0.874191403388977)]

### 5 - Visualizar agrupación de vectores

In [63]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [64]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

